In [ ]:
import boto3
from boto3.dynamodb.conditions import Key
import urllib3
import requests
from datetime import datetime, timedelta

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
table_name = ""
index_name = ""
auth_client_id = ""
auth_client_secret = ""
auth_audience = ""
auth_url = ""
tokenizer_url = ""


def startEncryption():
    encrypt = Encrypt()
    for client in encrypt.get_clients():
        encrypted_data = encrypt.encrypt_data()
        encrypt.update_encrypted_data(
            client["LoanNumber"], client["SortKey"], encrypted_data
        )


class Authenicated:
    __access_token = None
    __expiry_time = None

    def get_token(self):
        if (self.__access_token is None) or (self.__expiry_time <= datetime.now()):
            print("New token")
            request_body = {
                "client_id": auth_client_id,
                "client_secret": auth_client_secret,
                "audience": auth_audience,
                "grant_type": "client_credentials",
            }
            response = requests.post(sso_url, data=request_body, verify=False)
            if response.status_code == 200:
                jsonResp = response.json()
                expires_seconds = jsonResp["expires_in"]
                self.__access_token = jsonResp["access_token"]
                self.__expiry_time = datetime.now() + timedelta(seconds=expires_seconds)
                return self.__access_token
        else:
            print("reuse token")
            return self.__access_token


class Encrypt:
    def __init__(self):
        db = boto3.resource("dynamodb", region_name="us-east-2", verify=False)
        self.table = db.Table(table_name)

    def __get_records(self):
        last_key = None
        constId = ""
        condition = Key("LoanIdentifier").eq(f"{constId}") & Key("SortKey").begins_with(
            f"{constId}#"
        )
        while True:
            response = self.table.query(
                IndexName=index_name,
                KeyConditionExpression=condition,
                Limit=30,
                ExclusiveStartKey=last_key,
            )
            if response:
                items = response["Items"]
                for item in items:
                    col_val = item["Col"] if "Col" in item else ""
                    client = {
                        "LoanNumber": item["LoanNumber"],
                        "SortKey": item["SortKey"],
                        "Col": col_val,
                    }
                    yield client
                if "LastEvaluatedKey" in response:
                    break
                last_key = response["LastEvaluatedKey"]

    def get_clients(self):
        for client in self.__get_client_records():
            id = client["id"]
            if not id:
                continue
            else:
                yield client

    def encrypt_data(self, data):
        url = f"{tokenizer_url}/token/data"
        c = Authenicated()
        data = {"value": data}
        header = {"Authorization": c.get_token()}
        response = requests.post(url, data, headers=header, verify=False)
        if response.status_code == 200:
            content = response.json()
            return content["token"]
        return ""

    def update_encrypted_data(self, primaryKey, sortKey, encryptedValue):
        try:
            self.table.update_item(
                Key={"PrimaryKey": primaryKey, "SortKey": sortKey},
                UpdateExpression="set columnName=:colval, SchemaVersion=:v",
                ExpressionAttributeValues={":colval": encryptedValue, ":v": 4},
            )
        except ex as Exception:
            print(ex)
            # log the loan, sortkey, encrypteddata